In [1]:
# -*- coding: utf-8 -*-

import os
os.chdir('/home/zl/develop/projects/hbc')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, ensemble, metrics, grid_search, model_selection, decomposition, linear_model
import xgboost as xgb
from xgboost import XGBClassifier
from matplotlib.pyplot import rcParams
plt.rcParams['font.sans-serif']=['SimHei']

/home/zl/dev/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/zl/dev/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# load training&test set
df_train = pd.read_csv('./result/data_train_m.csv')
#df_train = pd.read_csv('./result/data_train.csv', encoding='gb2312')
#df_test = pd.read_csv('./result/data_test.csv', encoding='gb2312')
idcol = 'userid'
target = 'futureOrderType'
drop_tags = [idcol, target]
x_tags = [x for x in df_train.columns if x not in drop_tags]
y_tag = target

In [3]:
df_train.head()

,userid,futureOrderType,histord_sum_all,histord_sum_1,histord_ratio1_0,histord_time_last_1_month,histord_time_last_1_hour,histord_time_last_1_yday,histord_sum_cont1,histord_sum_cont2,...,印度尼西亚,吉隆坡,马德里,山西,重庆,北海道--札幌,湖南,江苏,新加坡.1,台北
0,100000000013,0,2.0,0.0,0.333333,8.0,3.0,218.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,100000000111,0,0.0,0.0,1.000000,-1.0,-1.0,-1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,100000000127,0,0.0,0.0,1.000000,-1.0,-1.0,-1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,100000000231,0,0.0,0.0,1.000000,-1.0,-1.0,-1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,100000000379,0,0.0,0.0,1.000000,-1.0,-1.0,-1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = np.array(df_train[x_tags])
y_train = np.array(df_train[y_tag])
#X_test = np.array(df_test[x_tags])
print(X_train.shape)

(40307, 136)


In [ ]:
def modelfit(alg, X_train, y_train, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_params()
        xgtrain = xgb.DMatrix(X_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds, metrics='auc', early_stopping_rounds=early_stopping_rounds)
        print('best n_estimators:', cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])
    # fit the algorithm on the data
    alg.fit(X_train, y_train, eval_metric='auc')
    # predict train set
    dtrain_pred = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    # print model report
    print('\nModel Report:')
    print("AUC Score (Train): %.5g" % metrics.roc_auc_score(y_train, dtrain_predprob))
    print("Accuracy: %.5g" % metrics.accuracy_score(y_train, dtrain_pred))

In [ ]:
### prediction model
np.random.seed(314)
dtrain = xgb.DMatrix(X_train, y_train)
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

num_round = 10
bst = xgb.train(dtrain=dtrain)

#model_xgb1 = xgb.XGBClassifier()
#modelfit(model_xgb1, X_train, y_train)
#model_xgb1.fit(, y_train)
#print('score_AUC:', round(metrics.roc_auc_score(y_train, model_xgb1.predict_proba(X_train)[:,1]), 5))
#print('score_precision:', round(metrics.accuracy_score(y_train, model_xgb1.predict(X_train)), 5))
#scores_cross = model_selection.cross_val_score(model_xgb1, X_train, y_train, cv=5, scoring='roc_auc')
#print('score_cross:', round(np.mean(scores_cross), 5), 'std:', round(np.std(scores_cross), 5))

In [ ]:
### final model
model_xgb_final = model_xgb1
#model_xgb_final.fit(X_train, y_train)
y_train_pred = model_xgb_final.predict(X_train)
y_train_predprob = model_xgb_final.predict_proba(X_train)[:,1]
importances = model_xgb_final.feature_importances_
df_featImp = pd.DataFrame({'tags': x_tags, 'importance': importances})
df_featImp_sorted = df_featImp.sort_values(by=['importance'], ascending=False)
#df_featImp_sorted.plot(x='tags', y='importance', kind='bar')
df_featImp_sorted.to_csv('./feature/feat_2018-01-07-1a.csv', index=False)

In [ ]:
featcount = 213
x_tags = df_featImp_sorted[:featcount]['tags'].tolist()
print(x_tags[-5:])

In [ ]:
# write out prediction result
y_test_pred = model_xgb_final.predict_proba(X_test)[:,1]

print('score_AUC:', round(metrics.roc_auc_score(y_train, y_train_predprob), 5))
print('score_precision:', round(metrics.accuracy_score(y_train, y_train_pred), 5))
scores_cross = model_selection.cross_val_score(model_xgb_final, X_train, y_train, cv=5, scoring='roc_auc')
print('score_cross:', round(np.mean(scores_cross), 5), 'std:', round(np.std(scores_cross), 5))

# write out prediction result
y_test_pred = model_xgb_final.predict_proba(X_test)[:,1]

df_profile = pd.read_csv('./data_train_test/userProfile_test.csv')
restable = pd.DataFrame(np.concatenate((np.array(df_profile['userid']).reshape((-1,1)), y_test_pred.reshape((-1,1))), axis=1))
restable.loc[:,0] = restable.loc[:,0].astype(np.int64)
pd.DataFrame(restable).to_csv("./result/orderFuture_test-20180104-2.csv", header=['userid','orderType'], index=False)